In [1]:
from pathlib import Path
from pylook.units import units
import pylook.calc as lc
import pylook_extra as lc2
import pylook_plotting_v4 as pl
import numpy as np
import pandas as pd

%matplotlib tk
# import matplotlib.pyplot as plt
# import matplotlib as mpl

In [2]:
data = lc2.readHDF("p5741_py.h5")

In [3]:
data['Time'] = np.cumsum(data['Time']/1e4)
data['time'] = data.pop('Time')

data['norm disp'] = data.pop('Hor DCDT')
data['norm stress'] = data.pop('Hor LOAD')
data['pc disp'] = data.pop('Pc Disp')
data['pc press'] = data.pop('Pc LOAD')
data['ppa disp'] = data.pop('Ppa Disp')
data['ppa press'] = data.pop('Ppa LOAD')
data['ppb press'] = data.pop('Ppb LOAD')
data['ppb disp'] = data.pop('Ppb Disp')
data['int disp'] = data.pop('Int DCDT')
data['ppa obdisp'] = data.pop('Ppa DCDT')
data['ppb obdisp'] = data.pop('Ppb DCDT')

In [4]:
# CALIBRATIONS FOR DISPLACEMENTS AND STRESSES
#---------------------------------------------------------------------------------------

# HORIZONTAL DISPLACEMENT
# high gain, short rod: 0.657956387503886 mm/V
h_disp_calib = (20/2**24) * 0.657956387503886 * 1000 #um/mm 
data['norm disp'] = data['norm disp'] * h_disp_calib

# HORIZONTAL INTERNAL DISPLACEMENT
# high gain: 0.415958632711548 mm/V
Hint_disp_calib = (20/2**24) * 0.415958632711548 * 1000 #um/mm 
data['int disp'] = data['int disp'] * Hint_disp_calib

# HORIZONTAL LOAD
# high gain: 129.9546436 mV/kN
# 1 surface
# area: 0.0022231311 m^2
h_load_calib = 1 * 0.0022292545 * 129.9546436 * (2**24/20)
data['norm stress'] = data['norm stress'] * 1/h_load_calib

#---------------------------------------------------------------------------------------

# Pc INTENSIFIER DISPLACEMNT
# high gain: 29.49852507 mm/V 
# ((20/2**24) * units('V / bit')) * (29.499 * units('mm / V') * 1000 * units('micron / mm'))
pc_disp_calib = (20/2**24) * 29.49852507 * 1000 #um/mm
data['pc disp'] = data['pc disp'] * pc_disp_calib

# Pc PRESSURE
# high gain: 0.14556041 V/MPa
pc_pres_calib = (20/2**24)  * 1/0.14556041
data['pc press'] = data['pc press'] * pc_pres_calib

#---------------------------------------------------------------------------------------
                                                     
# PpA INTENSIFIER DISPLACEMNT
# high gain: 26.73796791 mm/V 
ppa_disp_calib = (20/2**24) * 26.73796791 * 1000 #um/mm
data['ppa disp'] = data['ppa disp'] * ppa_disp_calib

# PpA PRESSURE
# high gain: 1.517680983 V/MPa
ppa_pres_calib = (20/2**24) * 1/1.517680983
data['ppa press'] = data['ppa press'] * ppa_pres_calib
ppa_obdisp_calib = (20/2**24) * 0.65 * 1000 #V/bit
data['ppa obdisp'] = data['ppa obdisp'] * ppa_obdisp_calib
                               
#---------------------------------------------------------------------------------------
                                                      
# PpB INTENSIFIER DISPLACEMNT
# high gain: 26.88172043 mm/V 
ppb_disp_calib = (20/2**24) * 26.88172043 * 1000 #um/mm
data['ppb disp'] = data['ppb disp'] * ppb_disp_calib
ppb_obdisp_calib = (20/2**24) * 0.65 * 1000 #V/bit
data['ppb obdisp'] = data['ppb obdisp'] * ppb_obdisp_calib

# PpB PRESSURE
# high gain: 1.48 V/MPa
ppb_pres_calib = (20/2**24) * 1/1.483019428
data['ppb press'] = data['ppb press'] * ppb_pres_calib
                                                      
#---------------------------------------------------------------------------------------                                                      

In [5]:
# pl.plotter(data, x=None, y='ppa obdisp', y2="ppb obdisp", plot_type="sharey", idx1=0, idx2=-1)

In [6]:
# OFFSETS AND ZERO POINTS

# HOR DISP
data['norm disp'] = lc.zero(data['norm disp'], 540, mode='before')

# INT HOR DISP
data['int disp'] = lc.zero(data['int disp'], 540, mode='before')

# HOR LOAD
data['norm stress'] = lc.zero(data['norm stress'], 540, mode='before')

# PC DISP
data['pc disp'] = lc.zero(data['pc disp'], 1347, mode='before')

# PC LOAD
data['pc press'] = lc.zero(data['pc press'], 1347, mode='before')

# Ppa LOAD
data['ppa press'] = lc.zero(data['ppa press'], 7180, mode='before')
data['ppa disp'] = lc.zero(data['ppa disp'], 7180, mode='before')
data['ppa obdisp'] = lc.zero(data['ppa obdisp'], 11073, mode='before')

# Ppb LOAD
data['ppb press'] = lc.zero(data['ppb press'], 7174, mode='before')
data['ppb disp'] = lc.zero(data['ppb disp'], 7174, mode='before')
data['ppb obdisp'] = lc.zero(data['ppb obdisp'], 20275, mode='before')

In [7]:
# pl('ppa disp', 'ppa obdisp', 0, -1)

In [8]:
##################################################################################
# cjm & cew; updated for True Triax config. 20181031                             #
#                                                                                #
#                                                                                #
#   In the vessel for SDS with notched L-shaped samples:                         #
#               Shear Stress is given by: tau = (FVp - Pc*Ap)/A                  #
#               Normal stress is given by: sigma_n = (FHp + Pc(A-Ap))/A          #
#                    or, sigma_n = FHp/A + Pc(A-Ap)/A                            #
#  where:                                                                        #
#  FVp is vertical force applied by the piston                                   #
#  FHp is horizontal force applied by the piston                                 #
#  A is the smaller area of the L-shaped samples                                 #
#   note that the frictional contact area will be given by the block thickness.  #
#  The eventual fracture plane will be given by the distance between the notches #
#  Ap is the area of the piston                                                  #
#  Pc is the confining pressure                                                  #
#                                                                                #
##################################################################################


# Account for Pc Force pushing on pistons
# Horizontal Area = 0.0022292545 m^2
A = 0.0022231311
#adjust normal stress for Pc.
#Area of piston, Ap = 44mm (dia) =  0.00152053084 m^2
Ap = 0.044 * np.pi
Pc_area = 100 - (Ap-A)/A
# ~ 32% of Pc is added to the applied horizontal stress to get the effective stress

# Calculate Effective Stresses
data['effNS'] = data['norm stress'] + (data['pc press'] * Pc_area/100) - (data['ppa press'] + data['ppb press'])/2

In [9]:
pl.plotter(data, x=None, y="ppb obdisp", idx1=51400, idx2=4503992, plot_type="xy")

In [10]:
# from corr_funcs import butter_filter

data['temp'] = lc2.butter_filter(data['ppb obdisp'], 15, 1000, 3, 'low')
# pl.plotter(data, x=None, y="ppb obdisp", y2="temp", idx1=51400, idx2=4503992, plot_type="sharey")

In [11]:
# pl.plotfr(data, "time", "temp", idx1=51400, idx2=4503992, Fs=1000, maglog=True)

In [12]:
data['ppb obdisp filt'] = lc2.butter_filter(data['ppb obdisp'], 13, 1000, 3, 'low')
data['ppa obdisp filt'] = lc2.butter_filter(data['ppa obdisp'], 13, 1000, 3, 'low')

In [13]:
data['ppb press filt'] = lc2.butter_filter(data['ppb press'], 20, 1000, 2, 'low')
data['ppa press filt'] = lc2.butter_filter(data['ppa press'], 20, 1000, 2, 'low')

In [14]:
# pl.plotter(data, "time", 'ppa press', 'ppa press filt', y2log=False, idx1=0, idx2=-1, plot_type="sharey")

In [15]:
# pl.plotfr(data, "time", "ppb press", 1000, 0, -1, maglog=True)

In [16]:
# pl.plotter(data, "time", 'ppa obdisp', 'ppa obdisp filt', y2log=False, idx1=0, idx2=-1, plot_type="sharey")

In [26]:
###############################################################################
# VOLUMETRIC FLOW                                                             #
# first, calculate displacement rate of the pistons with running avg. slope   #
# then, multiply by inner diameter of intensifiers for volumetric rate        #
###############################################################################

# sampfreq = 1000
# sampfreqidxs = np.where(np.logical_and(sampfreq < (sampfreq-10), sampfreq > (sampfreq+10)))
# data['PpDiff'][sampfreqidxs] = np.nan

###############################################################################

# calculate Pressure differential & convert from [MPa] --> [Pa]
data['PpDiff'] = (data['ppb press filt'] - data['ppa press filt']) * 1e6 #Pa

data['mmPpDiff'] = lc2.movingmean(data['PpDiff'], 1001)

# INTENSIFIER DISPLACEMENT RATES
data['AvPparate'] = lc2.movingslope(data['ppa disp'], 1001)
data['AvPpbrate'] = lc2.movingslope(data['ppb disp'], 1001)
data['AvPparate2'] = lc2.movingslope(data['ppa obdisp filt'], 1001)
data['AvPpbrate2'] = lc2.movingslope(data['ppb obdisp filt'], 1001)

# INTENSIFIER VOLUME
piston_dia = 0.0254 #meter
Vol_int = np.pi*(piston_dia/2)**2 #m^2

# FLOW RATES
data['Qa'] = data['AvPparate']/1e6 * Vol_int #m^3/s
data['Qb'] = -1*data['AvPpbrate']/1e6 * Vol_int #m^3/s
data['Qa2'] = -1*data['AvPparate2']/1e6 * Vol_int #m^3/s
data['Qb2'] = data['AvPpbrate2']/1e6 * Vol_int #m^3/s

#flow in this exp is from to B to A
#check to make sure we have steady-state flow -- percent difference 
data['QpctDiff'] = (data['Qb'] - data['Qa']) / data['Qb'] * 100 #percent
data['QpctDiff2'] = (data['Qb2'] - data['Qa2']) / data['Qb2'] * 100 #percent

#calculate an average flow rate from upsteam and downstream
data['Qavg'] = (data['Qa'] + data['Qb'])/2 #m^3/s
data['Qavg2'] = (data['Qa2'] + data['Qb2'])/2 #m^3/s

# Fracture plane length x thickness 45 x 23.43 = 0.00105435 m^2
A_flow = 0.045 * 0.02343 #m^2

# flow length is 0.04976 m
L_flow = 0.04976 #meter

#viscosity of water at 25 deg. C = 1e-3 Pa.s
nu = 1e-3 #Pa.s

# permeability calculation [m^2]
data['perm'] = (data['Qavg'] * nu * L_flow)/(data['mmPpDiff'] * A_flow) #[m^2]
data['perm2'] = (data['Qavg2'] * nu * L_flow)/(data['mmPpDiff'] * A_flow) #[m^2]

data["perm"][0:60600] = np.nan
data["perm"][4503975::] = np.nan
data["perm2"][0:60600] = np.nan
data["perm2"][4503975::] = np.nan

In [18]:
# pl.plotter(data, "time", 'Qa2', 'Qb2', y2log=False, idx1=60600, idx2=4503975, plot_type="sharey")

In [30]:
pl.plotter(data, x="time", y='PpDiff', y2='perm2', y2log=False, idx1=0, idx2=-1, plot_type="sub2")

In [20]:
# max_flow_diff_percent = 100
# # min_flow = 1; # in m^3/s. Minimum flow

# flow_diff_idxs = data['QpctDiff'].abs().to_numpy() > max_flow_diff_percent
# flow_diff_idxs2 = data['QpctDiff2'].abs().to_numpy() > max_flow_diff_percent
# # min_flow_idxs = np.logical_or(data['Qa'].to_numpy() <= min_flow, data['Qb'].to_numpy() >= min_flow)

# data['perm'].to_numpy()[flow_diff_idxs] = np.nan
# data['perm2'].to_numpy()[flow_diff_idxs2] = np.nan

In [21]:
data.drop(['Vert DCDT', 'Vert LOAD', 'PpDiff','mmPpDiff','AvPparate','AvPpbrate','QpctDiff','Qavg','AvPparate2','AvPpbrate2','QpctDiff2','Qavg2'], axis=1, inplace=True)

In [22]:
lc2.saveHDF("p5741_r", data)